This script can be used to generate the following events given some KPI constarints for one palnned date for one particular line:

*   FTT
*   Defective
*   Defects
*   Reject
*   Rectified
*   Manpower

The inputs we need are:

1. A plan for one day/ multipe days  for the line of interest
2. An hourly table of KPIs for each day (date column and hour column both)
3. General schedule (Assuming we reuse the same schedule for all days)
4. An example etmplate for each event type (just to order the columns)

We have all 4 sheets in one file  above in the link here which we will upload to **content/sample_data**
[link text](https://docs.google.com/spreadsheets/d/1sX9sf6zxVHKcaqhAdU-L36pvrUDtaOqK/edit?usp=drive_link)

 5. Synthesizers : Dump of all the pretrained synthesizers needed to generate  
    the constrained synthetic data.
    **(Upload to content/sample_data when running the script)**
[link text](https://drive.google.com/drive/folders/156GDl2-paytduo3noFF55BPhQzvFjBD_?usp=drive_link)


The output files per attribute will be dumped under **content/** using the attribute name in .csv format.




   









In [1]:
# All imports
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import string

In [2]:
# Read constraints sheet
xls = pd.read_excel("/content/sample_data/constraints_mul_days.xlsx",
                    sheet_name=["plan","schedule", "target_kpis",
                                "ftt", "defective", "reject", "defects",
                                "rectified", "manpower"])

# Access individual sheets using sheet names
plan = xls["plan"]
schedule = xls["schedule"]
target_kpis = xls["target_kpis"]
ftt = xls["ftt"]
defective = xls["defective"]
reject = xls["reject"]
defects = xls["defects"]
rectified = xls["rectified"]
manpower = xls["manpower"]

In [3]:
# Timezone of the factory
hour_offset = 5.5

I assume that the plan is uploaded per-day (can have multiple plans though)

In [4]:
# Function to generate random timestamps per-day that lies between a given start and end times

import random
import pandas as pd
from datetime import datetime, timedelta

def generate_random_timestamps(start_time, end_time, min_increment, max_increment, num_increments):
    """
    Generate a sorted list of timestamps with random increments for a specified date and time range.

    :param start_time: The start datetime.
    :param end_time: The end datetime.
    :param min_increment: Minimum increment in minutes.
    :param max_increment: Maximum increment in minutes.
    :param num_increments: Number of random increments to generate.
    :return: A DataFrame with the sorted timestamps.
    """
    # Generate random increments
    increments = [random.randint(min_increment, max_increment) for _ in range(num_increments)]

    # Generate timestamps
    timestamps = []
    current_time = start_time

    while current_time < end_time:
        timestamps.append(current_time)
        increment = timedelta(seconds=random.choice(increments))
        current_time += increment
        if current_time > end_time:
            break

    # Ensure timestamps are sorted
    timestamps.sort()

    # Create a DataFrame
    df = pd.DataFrame({'ts': timestamps})

    return df

In [5]:
all_dates = plan["date"].to_list()

In [6]:
# Logic to determine function parameters(with an offset for factory loacl time)

date_list =[]
start_time_list = []
end_time_list = []
num_increments_list = []

for index in range(len(all_dates)):
  date = all_dates[index].to_pydatetime().date()
  start_time_str = pd.to_timedelta(schedule["start_time"].iloc[0].strftime('%H:%M:%S'))- pd.Timedelta(hours=hour_offset)
  end_time_str = pd.to_timedelta(schedule["end_time"].iloc[-1].strftime('%H:%M:%S'))- pd.Timedelta(hours=hour_offset)

  start_time = datetime.combine(date, (datetime.min + start_time_str).time())
  end_time = datetime.combine(date, (datetime.min+ end_time_str).time())

  num_increments = plan["qty"].iloc[index] + random.randint(-1, 1)
  print(date)
  print(start_time)
  print(end_time)
  print(num_increments)

  date_list.append(date)
  start_time_list.append(start_time)
  end_time_list.append(end_time)
  num_increments_list.append(num_increments)



  #num_increments = 1000

min_increment = 2
max_increment = 6

2024-06-01
2024-06-01 02:00:00
2024-06-01 13:30:00
1101
2024-06-02
2024-06-02 02:00:00
2024-06-02 13:30:00
1000


In [7]:
# Verification
print(start_time_list)


[datetime.datetime(2024, 6, 1, 2, 0), datetime.datetime(2024, 6, 2, 2, 0)]


**FTT**

In [8]:
# Generate a vey large amount of timestamps(We want to make sure we generate
# more datapoints than the constrained amount so that we can randomly drop
# certain instances)
ts_df = pd.DataFrame()

for i in range(len(date_list)):
  ts_df= pd.concat([ts_df, generate_random_timestamps(start_time_list[i], end_time_list[i],
                                    min_increment,max_increment,
                                    num_increments_list[i])])
ts_df.reset_index(drop=True, inplace=True)
ts_df

,ts
0,2024-06-01 02:00:00
1,2024-06-01 02:00:02
2,2024-06-01 02:00:07
3,2024-06-01 02:00:10
4,2024-06-01 02:00:15
...,...
20677,2024-06-02 13:29:43
20678,2024-06-02 13:29:45
20679,2024-06-02 13:29:50
20680,2024-06-02 13:29:56


Now, I assume we have the trained synthesizers for each event type given below which we can load and use to generate more samples. But we may have to alter categorical data column values according to the plan's bctx and org. (So just learned neumerical distributions are learned.)

In [9]:
!pip install sdv

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [10]:
# To import a synthesizer
import sdv
from sdv.single_table import CTGANSynthesizer

In [11]:
# Import the pretrained synthesizer for FTT
ftt_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/ftt_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
# Generate some instances (to teh same length as timestamps)
synthetic_ftt = ftt_synthesizer.sample(num_rows=len(ts_df))

Sampling rows: 100%|██████████| 20682/20682 [00:05<00:00, 3936.86it/s]


In [13]:
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 12:33:28,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,7,0,0,0,0,0,0
1,2024-06-01 12:57:38,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,6,0,0,0,0,0,0
2,2024-06-01 06:52:09,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0
3,2024-06-01 06:28:48,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,8,0,0,0,0,0,0
4,2024-06-01 08:23:40,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0


In [14]:
# Function to determine the date based on a time threshold
def get_factory_date(dt):
    dt = dt + timedelta(hours=hour_offset)
    if dt.time() < pd.to_datetime('05:00:00').time():
        return (dt - timedelta(days=1)).date()
    else:
        return dt.date()

In [15]:
# Apply factory_date logic to evt_gmt
synthetic_ftt["evt_gmt"]= ts_df["ts"]

synthetic_ftt['evt_gmt'] = pd.to_datetime(synthetic_ftt['evt_gmt'])
synthetic_ftt['factory_date'] = synthetic_ftt['evt_gmt'].apply(get_factory_date)

In [16]:
# Verification
synthetic_ftt

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,7,0,0,0,0,0,0
1,2024-06-01 02:00:02,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,6,0,0,0,0,0,0
2,2024-06-01 02:00:07,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0
3,2024-06-01 02:00:10,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,8,0,0,0,0,0,0
4,2024-06-01 02:00:15,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,8,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20677,2024-06-02 13:29:43,2024-06-02,sdv-id-lBwJXf,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,6,0,0,0,0,0,0
20678,2024-06-02 13:29:45,2024-06-02,sdv-id-KkRPJX,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,6,0,0,0,0,0,0
20679,2024-06-02 13:29:50,2024-06-02,sdv-id-COVSop,1,Morning,1,mas,1,kreeda,ABC001,...,1,L,ftt,9,0,0,0,0,0,0
20680,2024-06-02 13:29:56,2024-06-02,sdv-id-mUnbfg,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,6,0,0,0,0,0,0


In [17]:
def edit_df_bctx_org(df,columns):
    # Ensure date_column is in datetime format
    df["factory_date"] = pd.to_datetime(df["factory_date"])
    plan["date"] = pd.to_datetime(plan["date"])

    # Iterate through each row in the df
    for i in range(len(df)):
        # Get the date for the current row in df
        current_date = df.loc[i, "factory_date"]

        # Find the corresponding row in the plan DataFrame
        plan_row = plan[plan["date"] == current_date]

        if not plan_row.empty:
            for column in columns:
                df.at[i, column] = plan_row.iloc[0][column]
        else:
            print(f"Date {current_date} not found in plan DataFrame")

    return df

In [18]:
# # The trained synthesizer will not have the same categorical
# # values as the current plan. we need to update them
# # Function to change bctx and org details
# def edit_df_bctx_org(df, columns):
#     for column in columns:
#       df[column] = plan[column].iloc[0]


In [19]:
# For the ease of manipulation, let us assume that we add piece by piece in
# all scenarios
# This functions sets a value column to a given value

def edit_value(df, column, value):
  df[column] = value

In [20]:
# # edit bctx and values as needed
# editable_columns = ["plan_id", "shift_id", "shift", "tenant_id", "tenant",
#                         "cluster_id", "cluster", "factory_id", "factory", "building_id",
#                         "building","floor_id", "floor", "zone_id", "zone", "section_id",
#                         "section", "module_id","module", "buyer_id","buyer","vendor_id",
#                         "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
#                         "fabric_type","product_category_id", "product_category",
#                         "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

# edit_df_bctx_org(synthetic_ftt, editable_columns)



In [21]:
edit_value(synthetic_ftt, "ftt_count", 1)

In [22]:
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
1,2024-06-01 02:00:02,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
2,2024-06-01 02:00:07,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
3,2024-06-01 02:00:10,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
4,2024-06-01 02:00:15,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0


In [23]:
def find_working_hour(column, hours):
  local_time = column + timedelta(hours=hours)
  return local_time.hour * 3600 + local_time.minute * 60 + local_time.second


In [24]:
synthetic_ftt['factory_time'] = synthetic_ftt['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,factory_time
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,27000
1,2024-06-01 02:00:02,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,27002
2,2024-06-01 02:00:07,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,27007
3,2024-06-01 02:00:10,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,27010
4,2024-06-01 02:00:15,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,27015


In [25]:
# Add the seconds of the day to the schedule df
schedule['start_time'] = pd.to_datetime(schedule['start_time'], format='%H:%M:00').dt.time
schedule['seconds_of_day'] = schedule['start_time'].apply(lambda x: x.hour * 3600 + x.minute * 60)

In [26]:
# Verification
schedule

,hour,start_time,end_time,seconds_of_day
0,1,07:30:00,08:30:00,27000
1,2,08:30:00,09:30:00,30600
2,3,09:30:00,10:30:00,34200
3,4,10:30:00,11:30:00,37800
4,5,11:30:00,13:00:00,41400
5,6,13:00:00,14:00:00,46800
6,7,14:00:00,15:00:00,50400
7,8,15:00:00,16:00:00,54000
8,9,16:00:00,17:00:00,57600
9,10,17:00:00,18:00:00,61200


In [27]:
# Perform a forward-looking merge
synthetic_ftt = synthetic_ftt.sort_values('factory_time')
synthetic_ftt = pd.merge_asof(synthetic_ftt, schedule, left_on='factory_time', right_on='seconds_of_day', direction='backward')
synthetic_ftt.drop(columns=['factory_time', 'start_time', 'end_time', 'seconds_of_day'], inplace=True)



In [28]:
# Verification
synthetic_ftt.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,hour
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,1
1,2024-06-02 02:00:00,2024-06-02,sdv-id-cJwXhe,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1
2,2024-06-01 02:00:02,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1
3,2024-06-02 02:00:04,2024-06-02,sdv-id-QAtwJB,1,Morning,1,mas,1,kreeda,ABC001,...,M,ftt,1,0,0,0,0,0,0,1
4,2024-06-01 02:00:07,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,ftt,1,0,0,0,0,0,0,1


In [29]:
synthetic_ftt = synthetic_ftt.rename(columns={'hour': 'Hour', 'factory_date': 'Date'})


In [30]:
target_kpis

,Date,Line,Hour,FTT,SMV,TPPM,Defective,Defect,Rectified,Reject,Operator,Helper,Man power
0,2024-06-01,MLF1001,1,91,8.28,1.666667,9,13,3,0,23,5,28
1,2024-06-01,MLF1001,2,95,8.28,1.666667,5,7,4,2,23,5,28
2,2024-06-01,MLF1001,3,95,8.28,1.666667,6,12,2,2,23,5,28
3,2024-06-01,MLF1001,4,92,8.28,1.666667,1,1,1,1,23,5,28
4,2024-06-01,MLF1001,5,93,8.28,1.666667,5,9,3,2,23,5,28
5,2024-06-01,MLF1001,6,91,8.28,1.666667,5,10,4,2,23,5,28
6,2024-06-01,MLF1001,7,90,8.28,1.666667,5,5,2,2,23,5,28
7,2024-06-01,MLF1001,8,93,8.28,1.666667,2,3,0,4,23,5,28
8,2024-06-01,MLF1001,9,92,8.28,1.666667,6,11,3,5,23,5,28
9,2024-06-01,MLF1001,10,94,8.28,1.666667,2,2,0,3,23,5,28


In [31]:
# Function to extract only the necessary attribute targets
def get_relevant_target(columns):
  targets = target_kpis[columns]
  return targets

In [32]:
# Traget for FTTs
targets_ftt = get_relevant_target(["Date","Hour", "FTT"])

In [33]:
# def adjust_hourly_counts(df, target_counts,count_column, hour_column='Hour'):
#     """
#     Adjust the counts in each hour group of a DataFrame to match the target counts.

#     :param df: DataFrame containing hour and count columns.
#     :param target_counts: DataFrame with hour and target_count columns.
#     :param hour_column: Name of the column containing hour values.
#     :param count_column: Name of the column containing count values.
#     :return: DataFrame with adjusted counts.
#     """
#     # Merge the dataframes on the 'hour' column
#     merged_df = df.merge(target_counts, on=hour_column, how='left')


#     # Create a list to store the adjusted groups
#     result_groups = []



#     # Iterate over each hour group
#     for hour, group in merged_df.groupby(hour_column):
#         # Calculate the difference between the current count and the target count
#         current_count = len(group)
#         target_count = group[count_column].iloc[0]
#         difference = current_count - target_count

#         # Drop rows randomly to meet the target count
#         if difference > 0:
#             drop_indices = group.sample(n=difference).index
#             group = group.drop(drop_indices)

#         # Append the group to the result list
#         result_groups.append(group)

#     # Concatenate the adjusted groups into a single DataFrame
#     result_df = pd.concat(result_groups, ignore_index=True)

#     # Drop the 'target_count' column as it's no longer needed
#     #result_df = result_df.drop(columns=[])

#     return result_df

In [34]:
def adjust_hourly_counts(df, target_counts, count_column, hour_column='Hour', date_column='Date'):
    """
    Adjust the counts in each hour and date group of a DataFrame to match the target counts.

    :param df: DataFrame containing hour, date, and count columns.
    :param target_counts: DataFrame with hour, date, and target_count columns.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with adjusted counts.
    """

    # Ensure date and hour columns are of the same type (datetime64[ns])
    df[date_column] = pd.to_datetime(df[date_column])  # Convert
    df[hour_column] = pd.to_datetime(df[hour_column], format='%H').dt.hour

    # Merge the dataframes on the 'hour' and 'date' columns
    merged_df = df.merge(target_counts, on=[hour_column, date_column], how='left')

    # Create a list to store the adjusted groups
    result_groups = []

    # Iterate over each hour and date group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Calculate the difference between the current count and the target count
        current_count = len(group)
        target_count = group[count_column].iloc[0]
        difference = current_count - target_count

        # Drop rows randomly to meet the target count
        if difference > 0:
            drop_indices = group.sample(n=difference).index
            group = group.drop(drop_indices)

        # Append the group to the result list
        result_groups.append(group)

    # Concatenate the adjusted groups into a single DataFrame
    result_df = pd.concat(result_groups, ignore_index=True)

    # Drop the 'target_count' column as it's no longer needed
    #result_df = result_df.drop(columns=[count_column])

    return result_df




In [35]:
synthetic_ftt_regulated = adjust_hourly_counts(synthetic_ftt, targets_ftt,"FTT", hour_column='Hour')
synthetic_ftt_regulated = synthetic_ftt_regulated.reindex(columns=ftt.columns)
synthetic_ftt_regulated['factory_date'] = synthetic_ftt_regulated['evt_gmt'].apply(get_factory_date)


In [36]:
# edit bctx and values as needed
editable_columns = ["plan_id", "shift_id", "shift", "tenant_id", "tenant",
                        "cluster_id", "cluster", "factory_id", "factory", "building_id",
                        "building","floor_id", "floor", "zone_id", "zone", "section_id",
                        "section", "module_id","module", "buyer_id","buyer","vendor_id",
                        "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                        "fabric_type","product_category_id", "product_category",
                        "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

edit_df_bctx_org(synthetic_ftt_regulated, editable_columns)

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size_id,size,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 02:00:02,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
1,2024-06-01 02:00:07,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
2,2024-06-01 02:03:14,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,1,L,ftt,1,0,0,0,0,0,0
3,2024-06-01 02:04:48,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,1,L,ftt,1,0,0,0,0,0,0
4,2024-06-01 02:07:03,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,2024-06-02 13:23:22,2024-06-02,13f0ffe7-c509-49b5-b203-83be908c4567,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
1973,2024-06-02 13:23:25,2024-06-02,13f0ffe7-c509-49b5-b203-83be908c4567,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
1974,2024-06-02 13:23:59,2024-06-02,13f0ffe7-c509-49b5-b203-83be908c4567,1,Morning,1,mas,1,kreeda,ABC001,...,1,M,ftt,1,0,0,0,0,0,0
1975,2024-06-02 13:25:15,2024-06-02,13f0ffe7-c509-49b5-b203-83be908c4567,1,Morning,1,mas,1,kreeda,ABC001,...,1,XL,ftt,1,0,0,0,0,0,0


In [37]:
synthetic_ftt_regulated.to_csv('synthetic_ftt_regulated.csv', index=False)

**DEFECTIVES**

In [38]:
# Generate a large number of timestamps
ts_defective = pd.DataFrame()

for i in range(len(date_list)):
  ts_defective= pd.concat([ts_defective, generate_random_timestamps(start_time_list[i], end_time_list[i],
                                    min_increment,max_increment,
                                    num_increments_list[i])])
ts_defective.reset_index(drop=True, inplace=True)
ts_defective.head()

,ts
0,2024-06-01 02:00:00
1,2024-06-01 02:00:02
2,2024-06-01 02:00:06
3,2024-06-01 02:00:08
4,2024-06-01 02:00:10


In [39]:
# # Generate a large number of timestamps
# ts_defective= generate_random_timestamps(start_time, end_time, min_increment,
#                                   max_increment, num_increments)
# ts_defective

In [40]:
# Import the pretrained synthesizer for defective
defective_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/defective_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [41]:
# Generate some instances (to the same length as timestamps)
synthetic_defective = defective_synthesizer.sample(num_rows=len(ts_defective))

Sampling rows: 100%|██████████| 20939/20939 [00:03<00:00, 6057.56it/s]


In [42]:
# Apply factory_date logic to evt_gmt
synthetic_defective["evt_gmt"]= ts_defective["ts"]

synthetic_defective['evt_gmt'] = pd.to_datetime(synthetic_defective['evt_gmt'])
synthetic_defective['factory_date'] = synthetic_defective['evt_gmt'].apply(get_factory_date)

In [43]:
# Edit defective value

edit_value(synthetic_defective, "defective_count", 1)

In [44]:
synthetic_defective.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-wnVsWx,0,1,1,0,0,0,0
1,2024-06-01 02:00:02,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-GrLoVq,0,1,1,0,0,0,0
2,2024-06-01 02:00:06,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-HRikVU,0,1,1,0,0,0,0
3,2024-06-01 02:00:08,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,sdv-id-DgKEuo,0,1,3,0,0,0,0
4,2024-06-01 02:00:10,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,L,defective,sdv-id-AQYBhy,0,1,1,0,0,0,0


In [45]:
# Function to generate gid
def generate_random_string(length=8):
    characters = string.ascii_uppercase + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

In [46]:
# Generate gid
synthetic_defective["gid"]= [generate_random_string() for _ in range(len(synthetic_defective))]

In [47]:
# Drop rows that may have generated the same random gid (rare but a precaution)
synthetic_defective= synthetic_defective.drop_duplicates(subset=['gid'])

In [48]:
synthetic_defective.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,YT72HHNS,0,1,1,0,0,0,0
1,2024-06-01 02:00:02,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,1402DIB4,0,1,1,0,0,0,0
2,2024-06-01 02:00:06,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,POSVJ71V,0,1,1,0,0,0,0
3,2024-06-01 02:00:08,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,XL,defective,0E4BENFM,0,1,3,0,0,0,0
4,2024-06-01 02:00:10,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,L,defective,9IG1YI7I,0,1,1,0,0,0,0


In [49]:
# Convert timestamp to seconds of day
synthetic_defective['factory_time'] = synthetic_defective['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
synthetic_defective.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,factory_time
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,defective,YT72HHNS,0,1,1,0,0,0,0,27000
1,2024-06-01 02:00:02,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,defective,1402DIB4,0,1,1,0,0,0,0,27002
2,2024-06-01 02:00:06,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,defective,POSVJ71V,0,1,1,0,0,0,0,27006
3,2024-06-01 02:00:08,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,defective,0E4BENFM,0,1,3,0,0,0,0,27008
4,2024-06-01 02:00:10,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,defective,9IG1YI7I,0,1,1,0,0,0,0,27010


In [50]:
# Perform a forward-looking merge
synthetic_defective = synthetic_defective.sort_values('factory_time')
synthetic_defective = pd.merge_asof(synthetic_defective, schedule, left_on='factory_time', right_on='seconds_of_day', direction='backward')
synthetic_defective.drop(columns=['factory_time', 'start_time', 'end_time', 'seconds_of_day'], inplace=True)
synthetic_defective = synthetic_defective.rename(columns={'hour': 'Hour', 'factory_date': 'Date' })
synthetic_defective.head()



,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,defective,YT72HHNS,0,1,1,0,0,0,0,1
1,2024-06-02 02:00:00,2024-06-02,sdv-id-IwWjBD,1,Morning,1,mas,1,kreeda,ABC001,...,defective,A61M9C6U,0,1,1,0,0,0,0,1
2,2024-06-01 02:00:02,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,defective,1402DIB4,0,1,1,0,0,0,0,1
3,2024-06-02 02:00:05,2024-06-02,sdv-id-wvHzSg,1,Morning,1,mas,1,kreeda,ABC001,...,defective,OJ7I6ZJM,0,1,3,0,0,0,0,1
4,2024-06-01 02:00:06,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,defective,POSVJ71V,0,1,1,0,0,0,0,1


In [51]:
# Target for defectives
targets_defective = get_relevant_target(["Date","Hour", "Defective", "Defect"])
targets_defective

,Date,Hour,Defective,Defect
0,2024-06-01,1,9,13
1,2024-06-01,2,5,7
2,2024-06-01,3,6,12
3,2024-06-01,4,1,1
4,2024-06-01,5,5,9
5,2024-06-01,6,5,10
6,2024-06-01,7,5,5
7,2024-06-01,8,2,3
8,2024-06-01,9,6,11
9,2024-06-01,10,2,2


In [52]:
pd.set_option('display.max_rows', None)

In [53]:
# Adjust the defective count to tally with the given target
defective_adjusted = adjust_hourly_counts(synthetic_defective, targets_defective,
                                          "Defective", hour_column="Hour")
defective_adjusted.head(20)
#defective_adjusted_final = defective_adjusted.drop(columns=['Hour', 'Defective',
                                                            #'Defect'])

,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour,Defective,Defect
0,2024-06-01 02:00:02,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,1,9,13
1,2024-06-01 02:01:17,2024-06-01,sdv-id-KzyMAD,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,1,9,13
2,2024-06-01 02:02:42,2024-06-01,sdv-id-SDbYPa,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,1,9,13
3,2024-06-01 02:05:16,2024-06-01,sdv-id-ryWKiB,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,1,9,13
4,2024-06-01 02:07:32,2024-06-01,sdv-id-SniZAu,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,1,9,13
5,2024-06-01 02:10:48,2024-06-01,sdv-id-QjfvOJ,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,1,9,13
6,2024-06-01 02:13:24,2024-06-01,sdv-id-SEUIBK,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,1,9,13
7,2024-06-01 02:40:07,2024-06-01,sdv-id-xpQNbr,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,3,0,0,0,0,1,9,13
8,2024-06-01 02:41:19,2024-06-01,sdv-id-mFuvnF,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,2,0,0,0,0,1,9,13
9,2024-06-01 03:01:25,2024-06-01,sdv-id-aXXtYD,1,Morning,1,mas,1,kreeda,ABC001,...,0,1,1,0,0,0,0,2,5,7


In [54]:
# Dataframe we can use to regulate defect counts
defective_adjusted.drop(columns=['Defective','Defect'], inplace=True)

In [55]:
def distribute_defects(total_defects, num_defectives):
  """
    Distribute a given number of total defects between a specified number
    of defective events.

    :param total_sum: The total defects to be distributed.
    :param num_events: The number of defective events.
    :return: A list with the distributed values.
  """
  # Initial distribution: assign each event a base value

  base_defects = total_defects // num_defectives
  remaining = total_defects % num_defectives

  distribution = [base_defects] * num_defectives

  # Randomly distribute the remaining value
  for _ in range(remaining):
      distribution[np.random.randint(0, num_defectives)] += 1

  return distribution



In [56]:
def generate_hourly_counts(defectives, targets, hour_column='Hour',date_column= 'Date', count_column='defect_count'):
    """
    Generate counts for each hour of each day group of a DataFrame to match the target counts by
    distributing the target sum among all rows in the hour + date group.

    :param defectives: DataFrame containing hour and count columns.
    :param targets: DataFrame with hour and target_count columns.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with generated counts.
    """
    # Merge the dataframes on the 'hour and date' column
    merged_df = defectives.merge(targets, on=[date_column, hour_column], how='left')

    # Create a new column for generated counts
    merged_df['generated_count'] = 0

    # Iterate over each hour group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Get the target sum for this date + hour
        target_sum = group['Defect'].iloc[0]

        # Generate the new counts by distributing the target sum
        generated_counts = distribute_defects(target_sum, len(group))
        merged_df.loc[group.index, 'generated_count'] = generated_counts

    # # Drop the 'target_count' column as it's no longer needed
    # result_df = merged_df.drop(columns=['target_count'])

    return merged_df


In [57]:
merged_df = generate_hourly_counts(defective_adjusted, targets_defective)
merged_df['defect_count']= merged_df['generated_count']



In [58]:
merged_df= merged_df.rename(columns={'Date': 'factory_date'})

In [59]:
# edit bctx and values as needed
editable_columns = ["plan_id", "shift_id", "shift", "tenant_id", "tenant",
                        "cluster_id", "cluster", "factory_id", "factory", "building_id",
                        "building","floor_id", "floor", "zone_id", "zone", "section_id",
                        "section", "module_id","module", "buyer_id","buyer","vendor_id",
                        "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                        "fabric_type","product_category_id", "product_category",
                        "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

synthetic_defective_regulated = edit_df_bctx_org(merged_df, editable_columns)

defects_for_defect = synthetic_defective_regulated.drop(columns=['generated_count','Defective', 'Defect'])


merged_df = synthetic_defective_regulated.drop(columns=['generated_count','Hour', 'Defective', 'Defect'])

merged_df.to_csv('synthetic_defective_regulated.csv', index=False)

**DEFECTS**

In [60]:
# To create the defects dataframe, explode each of the  events in defective data,
# defect_count times
def generate_defects_base(columns):
  df_repeated = pd.DataFrame()
  for col in columns:
    repeated_df = []
    for index, row in merged_df.iterrows():
        repeated_df.extend([row[col]] * row['defect_count'])
    df_repeated[col] = repeated_df
  return df_repeated

In [61]:
columns_for_defects = ["evt_gmt",	"factory_date",	"shift_id",	"shift",
                        "tenant_id",	"tenant",	"cluster_id",	"cluster",
                        "factory_id",	"factory",	"building_id",	"building",
                        "department_id",	"department",	"floor_id",	"floor",
                        "zone_id",	"zone",	"section_id","section",	"module_id",
                        "module","stp","size",	"hsk",
                        "buf","apt","stpv","buyer_id","buyer","vendor_id",
                        "vendor","po_id","po","so_id","so","li_id","li",
                        "fabric_type_id",	"fabric_type",	"product_category_id",
                        "product_category",	"product_type_id",	"product_type",
                        "style_id",	"style","color_id",	"color","size_id","gid"]
defects_base = generate_defects_base(columns_for_defects)

In [62]:
# Findout already processed columns
added_columns = defects_base.columns

In [63]:
# Findout what other columns should be in defects data

for column_name in added_columns:
    defects_temp = defects.drop(column_name, axis=1)
defects_temp.head(200)

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,issue_color_code,issue,operation_id,operation_code,operation,image,silhouette_id,silhouette,x,y
0,2024-06-01 02:15:00,2024-06-01 08:12:15,1,Morning,1,mas,1,kreeda,ABC011,ABC Factory,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,0.76,0.49
1,2024-06-02 02:16:00,2024-06-01 08:33:59,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#808080,Bobbling,18,117,Att Match Bow at Waist,front,1,shirt_style1,0.73,0.59
2,2024-06-02 02:16:00,2024-06-01 08:59:45,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#FFFFCC,Hi Low,11,110,Att Elastic at Both Legs,back,1,shirt_style1,1.00,0.07
3,2024-06-03 02:34:00,2024-06-01 08:59:45,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,7,106,BT at Waist W /Trim * 1,front,1,shirt_style1,1.00,0.75
4,2024-06-04 02:35:00,2024-06-01 08:59:45,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,1.00,0.00
5,2024-06-05 02:41:00,2024-06-01 08:32:28,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800080,Wavy,11,110,Att Elastic at Both Legs,front,1,shirt_style1,0.84,0.45
6,2024-06-06 02:43:00,2024-06-01 08:32:28,1,Morning,1,mas,1,kreeda,ABC011,ABC Factory,...,#000080,Pleat,17,116,Att Bow at CF Waist W / mark,back,1,shirt_style1,0.74,0.00
7,2024-06-07 02:45:00,2024-06-01 08:48:50,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#660066,Down Stc,11,110,Att Elastic at Both Legs,back,1,shirt_style1,1.00,0.64
8,2024-06-07 02:45:00,2024-06-01 08:48:50,1,Morning,1,mas,1,kreeda,ABC011,ABC Factory,...,#993366,Oil Spot,14,113,Att FO Elastic Around Waist,front,1,shirt_style1,0.87,0.45
9,2024-06-07 02:45:00,2024-06-01 08:48:50,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,7,106,BT at Waist W /Trim * 1,front,1,shirt_style1,0.77,0.40


In [64]:
# Import the pretrained synthesizer for defective
defects_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/defects_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [65]:
# Generate synthetic dataset that is the same length as aggregated defective data
synthetic_defects = defects_synthesizer.sample(num_rows=len(defects_base))

Sampling rows: 100%|██████████| 143/143 [00:00<00:00, 928.11it/s]


In [66]:
#Concatenate the aggregated defective data with generated defect-specific data
synthetic_defects = pd.concat([defects_base, synthetic_defects], axis=1)

In [67]:
# Reorder the data columns
synthetic_defects = synthetic_defects.reindex(columns=defects.columns)
synthetic_defects.head(20)

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,issue_color_code,issue,operation_id,operation_code,operation,image,silhouette_id,silhouette,x,y
0,2024-06-01 02:00:02,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#CCCCFF,Join Stc,11,110,Att Elastic at Both Legs,front,1,shirt_style1,0.98,0.35
1,2024-06-01 02:01:17,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#808000,Steps,11,110,Att Elastic at Both Legs,back,1,shirt_style1,0.52,1.00
2,2024-06-01 02:01:17,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#993366,Oil Spot,10,109,Join Bk Gusset Seam,back,1,shirt_style1,0.77,0.59
3,2024-06-01 02:02:42,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#FF00FF,Reverse,12,111,Join 1st Side Seam,front,1,shirt_style1,0.75,0.63
4,2024-06-01 02:05:16,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,0.92,0.75
5,2024-06-01 02:05:16,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#0066CC,Bow Defect,12,111,Join 1st Side Seam,back,1,shirt_style1,0.86,0.40
6,2024-06-01 02:07:32,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#993366,Oil Spot,14,113,Att FO Elastic Around Waist,back,1,shirt_style1,0.82,0.24
7,2024-06-01 02:10:48,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800080,Wavy,10,109,Join Bk Gusset Seam,back,1,shirt_style1,0.45,0.20
8,2024-06-01 02:10:48,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#800000,Lace Defect,13,112,Tack & Att label at CB Waist,back,1,shirt_style1,1.00,0.73
9,2024-06-01 02:13:24,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,#008080,Bartack,1,100,Join Fnt Gusset Seam,front,1,shirt_style1,0.69,0.13


In [68]:
synthetic_defects["evn"]= "defect"

In [69]:
synthetic_defects.to_csv('sythetic_defects_regulated.csv', index=False)

**RECTIFIED**

In [70]:
# We are going to derive rectified as portion of events from the defectives
# with the working hour attached.
defectives = defects_for_defect.copy()
defectives.head()
defectives.rename(columns={'factory_date': 'Date'}, inplace=True)

In [71]:
# Target for rectified
targets_rectified = get_relevant_target(["Date", "Hour", "Rectified"])
targets_rectified

,Date,Hour,Rectified
0,2024-06-01,1,3
1,2024-06-01,2,4
2,2024-06-01,3,2
3,2024-06-01,4,1
4,2024-06-01,5,3
5,2024-06-01,6,4
6,2024-06-01,7,2
7,2024-06-01,8,0
8,2024-06-01,9,3
9,2024-06-01,10,0


In [72]:
def pick_rectified_events(all_defective, targets, hour_column='Hour',date_column = 'Date', datetime_column='evt_gmt', count_column='Rectified'):
    """
    Pick the first n events from the raw events DataFrame for each hour group based on the counts DataFrame.

    :param all_defective: DataFrame containing raw events with an hour and datetime column.
    :param targets: DataFrame containing the count of events per hour.
    :param hour_column: Name of the column containing hour values.
    :param date_column: Name of the column containing date values.
    :param datetime_column: Name of the column containing datetime values.
    :param count_column: Name of the column containing count values.
    :return: DataFrame with the selected events.
    """
    # Ensure the datetime column is of datetime type
    all_defective[datetime_column] = pd.to_datetime(all_defective[datetime_column])

    # Merge the raw_df and counts_df on the date + hour column
    merged_df = all_defective.merge(targets, on=[date_column, hour_column], how='left')

    # Initialize an empty DataFrame to store the selected events
    selected_events = pd.DataFrame(columns=all_defective.columns)

    # Iterate over each date+hour group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Get the target sum for this date + hour
        target_sum = group['defect_count'].iloc[0]

    # Iterate over each hour group
    for (date, hour), group in merged_df.groupby([date_column, hour_column]):
        # Get the target count for this hour
        target_count = group[count_column].iloc[0]

        # Sort the group by the datetime column
        sorted_group = group.sort_values(by=datetime_column)

        # Select the first n events
        selected_n_events = sorted_group.head(target_count)

        # Append the selected events to the result DataFrame
        selected_events = pd.concat([selected_events, selected_n_events])

    # Drop the count column from the result DataFrame
    #selected_events = selected_events.drop(columns=[count_column])

    return selected_events


In [73]:
# Select certain defectives and transform them to rectified events
rectified_events = pick_rectified_events(defectives, targets_rectified)
rectified_events.drop(columns=['Hour', 'Rectified'], inplace=True)
rectified_events['evn'] = 'rectified'
rectified_events['defective_count'] = 0
rectified_events['defect_count'] = 0
rectified_events['rectified_count'] = 1



<ipython-input-72-ee0b19bcb418>:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  selected_events = pd.concat([selected_events, selected_n_events])


In [74]:
rectified_events.head()

,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,evn,gid,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 02:00:02,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,XL,rectified,1402DIB4,0,0,0,0,1,0,0
1,2024-06-01 02:01:17,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,L,rectified,56QLAUWL,0,0,0,0,1,0,0
2,2024-06-01 02:02:42,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,L,rectified,7F9HH60L,0,0,0,0,1,0,0
9,2024-06-01 03:01:25,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,XL,rectified,QHW6X7K7,0,0,0,0,1,0,0
10,2024-06-01 03:29:38,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,L,rectified,H8VYCW99,0,0,0,0,1,0,0


In [75]:
# Function to increment a timestamp by 10-20 seconds randomly
def increment_timestamp(timestamp):
    random_seconds = random.randint(1, 5)
    return timestamp + timedelta(seconds=random_seconds)

In [76]:
# Apply the function to the 'timestamp' column
rectified_events['evt_gmt'] = rectified_events['evt_gmt'].apply(increment_timestamp)

In [77]:
# Reorder the data columns
synthetic_rectified = rectified_events.reindex(columns=rectified.columns)


In [78]:
synthetic_rectified.rename(columns={'Date': 'factory_date'}, inplace=True)
synthetic_rectified.to_csv('synthetic_rectified_regulated.csv', index=False)

**REJECT**

In [79]:
# Generate a vey large amount of timestamps(We want to make sure we generate
# more datapoints than the constrained amount so that we can randomly drop
# certain instances)

ts_df_reject = pd.DataFrame()

for i in range(len(date_list)):
  ts_df_reject= pd.concat([ts_df_reject, generate_random_timestamps(start_time_list[i], end_time_list[i],
                                    min_increment,max_increment,
                                    num_increments_list[i])])
ts_df_reject.reset_index(drop=True, inplace=True)
ts_df_reject.head()

,ts
0,2024-06-01 02:00:00
1,2024-06-01 02:00:04
2,2024-06-01 02:00:06
3,2024-06-01 02:00:11
4,2024-06-01 02:00:15


In [80]:
# Import the pretrained synthesizer for Reject
reject_synthesizer = CTGANSynthesizer.load(
    filepath='/content/sample_data/reject_synthesizer.pkl'
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [81]:
# Generate some instances (to teh same length as timestamps)
synthetic_reject = reject_synthesizer.sample(num_rows=len(ts_df_reject))

Sampling rows: 100%|██████████| 20787/20787 [00:03<00:00, 6731.40it/s]


In [82]:
# Apply factory_date logic to evt_gmt
synthetic_reject["evt_gmt"]= ts_df_reject["ts"]

synthetic_reject['evt_gmt'] = pd.to_datetime(synthetic_reject['evt_gmt'])
synthetic_reject['factory_date'] = synthetic_reject['evt_gmt'].apply(get_factory_date)

In [83]:
synthetic_reject.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-vesfmW,reject,0,0,0,1,0,0,0
1,2024-06-01 02:00:04,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-tRzFCM,reject,0,0,0,1,0,0,0
2,2024-06-01 02:00:06,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-zMJmno,reject,0,0,0,1,0,0,0
3,2024-06-01 02:00:11,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-nATVMQ,reject,0,0,0,1,0,0,0
4,2024-06-01 02:00:15,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,XL,sdv-id-ZNygiF,reject,0,0,0,1,0,0,0


In [84]:
# Generate gid
synthetic_reject["gid"]= [generate_random_string() for _ in range(len(synthetic_reject))]

In [85]:
synthetic_reject['factory_time'] = synthetic_reject['evt_gmt'].apply(find_working_hour, args=(hour_offset,))
synthetic_reject.head()

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,factory_time
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,24CH92UT,reject,0,0,0,1,0,0,0,27000
1,2024-06-01 02:00:04,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,5TWP7W9Y,reject,0,0,0,1,0,0,0,27004
2,2024-06-01 02:00:06,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,6DX26DTH,reject,0,0,0,1,0,0,0,27006
3,2024-06-01 02:00:11,2024-06-01,sdv-id-jGEmvR,1,Morning,1,mas,1,kreeda,ABC001,...,YLWPHGQ6,reject,0,0,0,1,0,0,0,27011
4,2024-06-01 02:00:15,2024-06-01,sdv-id-agVyXC,1,Morning,1,mas,1,kreeda,ABC001,...,42VOXIWM,reject,0,0,0,1,0,0,0,27015


In [86]:
# Perform a forward-looking merge
synthetic_reject = synthetic_reject.sort_values('factory_time')
synthetic_reject = pd.merge_asof(synthetic_reject, schedule, left_on='factory_time', right_on='seconds_of_day', direction='backward')
synthetic_reject.drop(columns=['factory_time', 'start_time', 'end_time', 'seconds_of_day'], inplace=True)
synthetic_reject = synthetic_reject.rename(columns={'hour': 'Hour', 'factory_date': 'Date'})
synthetic_reject.head()




,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour
0,2024-06-01 02:00:00,2024-06-01,sdv-id-ZmALYB,1,Morning,1,mas,1,kreeda,ABC001,...,24CH92UT,reject,0,0,0,1,0,0,0,1
1,2024-06-02 02:00:00,2024-06-02,sdv-id-hsbspA,1,Morning,1,mas,1,kreeda,ABC001,...,IZI462C6,reject,0,0,0,1,0,0,0,1
2,2024-06-01 02:00:04,2024-06-01,sdv-id-ZhJSKp,1,Morning,1,mas,1,kreeda,ABC001,...,5TWP7W9Y,reject,0,0,0,1,0,0,0,1
3,2024-06-01 02:00:06,2024-06-01,sdv-id-RnJjTg,1,Morning,1,mas,1,kreeda,ABC001,...,6DX26DTH,reject,0,0,0,1,0,0,0,1
4,2024-06-02 02:00:06,2024-06-02,sdv-id-GTxxSo,1,Morning,1,mas,1,kreeda,ABC001,...,4VDSLR32,reject,0,0,0,1,0,0,0,1


In [87]:
# Target for defectives
targets_reject = get_relevant_target(["Date", "Hour", "Reject"])
targets_reject

,Date,Hour,Reject
0,2024-06-01,1,0
1,2024-06-01,2,2
2,2024-06-01,3,2
3,2024-06-01,4,1
4,2024-06-01,5,2
5,2024-06-01,6,2
6,2024-06-01,7,2
7,2024-06-01,8,4
8,2024-06-01,9,5
9,2024-06-01,10,3


In [88]:
# Adjust the reject count to tally with the given target
reject_adjusted = adjust_hourly_counts(synthetic_reject, targets_reject,
                                          "Reject", hour_column="Hour")
reject_adjusted.head(20)

,evt_gmt,Date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count,Hour,Reject
0,2024-06-01 03:04:19,2024-06-01,sdv-id-AACmfK,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,2,2
1,2024-06-01 03:37:59,2024-06-01,sdv-id-uEdOlJ,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,2,2
2,2024-06-01 04:25:26,2024-06-01,sdv-id-eqPUti,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,3,2
3,2024-06-01 04:41:41,2024-06-01,sdv-id-ZCJmzZ,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,3,2
4,2024-06-01 05:49:19,2024-06-01,sdv-id-GjkLHe,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,4,1
5,2024-06-01 06:12:19,2024-06-01,sdv-id-tZlySF,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,5,2
6,2024-06-01 06:16:03,2024-06-01,sdv-id-ZyhQzp,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,5,2
7,2024-06-01 07:50:56,2024-06-01,sdv-id-CVKaqR,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,6,2
8,2024-06-01 08:25:56,2024-06-01,sdv-id-ukbMuR,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,6,2
9,2024-06-01 09:16:13,2024-06-01,sdv-id-ChLVDe,1,Morning,1,mas,1,kreeda,ABC001,...,reject,0,0,0,1,0,0,0,7,2


In [89]:
reject_adjusted.rename(columns={'Date': 'factory_date'}, inplace=True)
synthetic_reject_regulated= reject_adjusted.reindex(columns=reject.columns)
synthetic_reject_regulated.head()



,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 03:04:19,2024-06-01,sdv-id-AACmfK,1,Morning,1,mas,1,kreeda,ABC001,...,XL,PTD4FH3I,reject,0,0,0,1,0,0,0
1,2024-06-01 03:37:59,2024-06-01,sdv-id-uEdOlJ,1,Morning,1,mas,1,kreeda,ABC001,...,M,GELX0XJX,reject,0,0,0,1,0,0,0
2,2024-06-01 04:25:26,2024-06-01,sdv-id-eqPUti,1,Morning,1,mas,1,kreeda,ABC001,...,XL,T6NK2AF5,reject,0,0,0,1,0,0,0
3,2024-06-01 04:41:41,2024-06-01,sdv-id-ZCJmzZ,1,Morning,1,mas,1,kreeda,ABC001,...,XL,E0XRD252,reject,0,0,0,1,0,0,0
4,2024-06-01 05:49:19,2024-06-01,sdv-id-GjkLHe,1,Morning,1,mas,1,kreeda,ABC001,...,M,ZQLJNJCG,reject,0,0,0,1,0,0,0


In [90]:
# edit bctx and values as needed
editable_columns = ["plan_id", "shift_id", "shift", "tenant_id", "tenant",
                        "cluster_id", "cluster", "factory_id", "factory", "building_id",
                        "building","floor_id", "floor", "zone_id", "zone", "section_id",
                        "section", "module_id","module", "buyer_id","buyer","vendor_id",
                        "vendor", "po_id", "po", "so_id","so","li_id","li","fabric_type_id",
                        "fabric_type","product_category_id", "product_category",
                        "product_type_id", "product_type", "style_id", "style", "color_id", "color" ]

edit_df_bctx_org(synthetic_reject_regulated, editable_columns)

edit_value(synthetic_reject_regulated, "reject_count", 1)

In [91]:
synthetic_reject_regulated.head(10)

,evt_gmt,factory_date,plan_id,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,...,size,gid,evn,ftt_count,defective_count,defect_count,reject_count,rectified_count,input_count,rw_defective_count
0,2024-06-01 03:04:19,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,XL,PTD4FH3I,reject,0,0,0,1,0,0,0
1,2024-06-01 03:37:59,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,M,GELX0XJX,reject,0,0,0,1,0,0,0
2,2024-06-01 04:25:26,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,XL,T6NK2AF5,reject,0,0,0,1,0,0,0
3,2024-06-01 04:41:41,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,XL,E0XRD252,reject,0,0,0,1,0,0,0
4,2024-06-01 05:49:19,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,M,ZQLJNJCG,reject,0,0,0,1,0,0,0
5,2024-06-01 06:12:19,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,XL,MEI2CUCL,reject,0,0,0,1,0,0,0
6,2024-06-01 06:16:03,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,XL,BTA0Y1B1,reject,0,0,0,1,0,0,0
7,2024-06-01 07:50:56,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,XL,EFS6UKVS,reject,0,0,0,1,0,0,0
8,2024-06-01 08:25:56,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,XL,MJMRVAOM,reject,0,0,0,1,0,0,0
9,2024-06-01 09:16:13,2024-06-01,12f0ffe7-c509-49b5-b203-83be908b2883,1,Morning,1,mas,1,kreeda,ABC001,...,L,0KFQ6PFD,reject,0,0,0,1,0,0,0


In [92]:
synthetic_reject_regulated.to_csv('synthetic_reject_regulated.csv', index=False)

**MANPOWER**

Generate categorical data columns using for example, ftt events and generate the numbers directly using schedule.

In [93]:
schedule

,hour,start_time,end_time,seconds_of_day
0,1,07:30:00,08:30:00,27000
1,2,08:30:00,09:30:00,30600
2,3,09:30:00,10:30:00,34200
3,4,10:30:00,11:30:00,37800
4,5,11:30:00,13:00:00,41400
5,6,13:00:00,14:00:00,46800
6,7,14:00:00,15:00:00,50400
7,8,15:00:00,16:00:00,54000
8,9,16:00:00,17:00:00,57600
9,10,17:00:00,18:00:00,61200


In [94]:
def expand_schedule_with_dates(schedule_df, date_list):
    """
    Repeat the schedule DataFrame for each date in the date_list and add the relevant date.

    :param schedule_df: The schedule DataFrame with 11 hours.
    :param date_list: List of dates to repeat the schedule for.
    :return: A DataFrame with the schedule repeated for each date, with the date added to each row.
    """
    # Create an empty DataFrame to store the results
    expanded_df = pd.DataFrame()

    # Repeat the schedule DataFrame for each date
    for date in date_list:
        temp_df = schedule_df.copy()  # Copy the schedule DataFrame
        temp_df['Date'] = date  # Add the date column
        expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)

    return expanded_df

In [95]:
new_schedule = expand_schedule_with_dates(schedule, date_list)
new_schedule

,hour,start_time,end_time,seconds_of_day,Date
0,1,07:30:00,08:30:00,27000,2024-06-01
1,2,08:30:00,09:30:00,30600,2024-06-01
2,3,09:30:00,10:30:00,34200,2024-06-01
3,4,10:30:00,11:30:00,37800,2024-06-01
4,5,11:30:00,13:00:00,41400,2024-06-01
5,6,13:00:00,14:00:00,46800,2024-06-01
6,7,14:00:00,15:00:00,50400,2024-06-01
7,8,15:00:00,16:00:00,54000,2024-06-01
8,9,16:00:00,17:00:00,57600,2024-06-01
9,10,17:00:00,18:00:00,61200,2024-06-01


In [96]:
manpower_cat = synthetic_ftt_regulated.head(len(new_schedule))
manpower_cat.drop(columns=['plan_id', 'evn', 'ftt_count', 'defective_count', 'defect_count',
                           'reject_count', 'rectified_count', 'input_count',
                           'rw_defective_count', 'buyer_id','buyer','vendor_id',
                           'vendor',	'po_id'	,'po',	'so_id',	'so',	'li_id'	,
                           'li',	'fabric_type_id',	'fabric_type',	'product_category_id',
                           'product_category',	'product_type_id',	'product_type',
                           'style_id',	'style',	'color_id',	'color'	,'size_id',	'size'], inplace=True)


In [97]:
# Iterate over each datetime object in the Series and format it
start_time_str = [dt.strftime('%H:%M:%S') for dt in new_schedule["start_time"]]

ts_manpower = pd.DataFrame()
ts_manpower['start_time'] = start_time_str
ts_manpower





,start_time
0,07:30:00
1,08:30:00
2,09:30:00
3,10:30:00
4,11:30:00
5,13:00:00
6,14:00:00
7,15:00:00
8,16:00:00
9,17:00:00


In [98]:
# Generate timestamps for manpower events
manpower_cat['evt_gmt'] = pd.to_datetime(new_schedule['Date'].astype(str) + ' ' +ts_manpower['start_time'] )
manpower_cat

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,section,module_id,module,station_id,station,hsk,buf,apt,stp,stpv
0,2024-06-01 07:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,Section 1,MLF1001,Line 1,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0
1,2024-06-01 08:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,Section 1,MLF1001,Line 1,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0
2,2024-06-01 09:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,Section 1,MLF1001,Line 1,EQF1220,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0
3,2024-06-01 10:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,Section 1,MLF1001,Line 1,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0
4,2024-06-01 11:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,Section 1,MLF1001,Line 1,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0
5,2024-06-01 13:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,Section 1,MLF1001,Line 1,EQF1219,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0
6,2024-06-01 14:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,Section 1,MLF1001,Line 1,EQF1220,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0
7,2024-06-01 15:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,Section 1,MLF1001,Line 1,EQF1219,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0
8,2024-06-01 16:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,Section 1,MLF1001,Line 1,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0
9,2024-06-01 17:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,Section 1,MLF1001,Line 1,EQF1219,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0


In [99]:
# Generate Hour
manpower_cat['Hour'] = new_schedule['hour']
manpower_cat


,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,module_id,module,station_id,station,hsk,buf,apt,stp,stpv,Hour
0,2024-06-01 07:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,MLF1001,Line 1,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,1
1,2024-06-01 08:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,MLF1001,Line 1,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,2
2,2024-06-01 09:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,MLF1001,Line 1,EQF1220,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,3
3,2024-06-01 10:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,MLF1001,Line 1,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,4
4,2024-06-01 11:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,MLF1001,Line 1,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,5
5,2024-06-01 13:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,MLF1001,Line 1,EQF1219,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,6
6,2024-06-01 14:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,MLF1001,Line 1,EQF1220,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,7
7,2024-06-01 15:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,MLF1001,Line 1,EQF1219,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,8
8,2024-06-01 16:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,MLF1001,Line 1,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,9
9,2024-06-01 17:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,MLF1001,Line 1,EQF1219,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,10


In [100]:
# Target for defectives (This example target does not have ironman and qc)
targets_manpower = get_relevant_target(["Date", "Hour", "Operator", "Helper", "Man power"
                                        #"Iron man", "QC"
                                        ])
targets_manpower

,Date,Hour,Operator,Helper,Man power
0,2024-06-01,1,23,5,28
1,2024-06-01,2,23,5,28
2,2024-06-01,3,23,5,28
3,2024-06-01,4,23,5,28
4,2024-06-01,5,23,5,28
5,2024-06-01,6,23,5,28
6,2024-06-01,7,23,5,28
7,2024-06-01,8,23,5,28
8,2024-06-01,9,23,5,28
9,2024-06-01,10,23,5,28


In [101]:
# lookup the hourly values from given target
manpower_cat.rename(columns={'factory_date': 'Date'}, inplace=True)
synthetic_manpower = manpower_cat.merge(targets_manpower, on=["Date", 'Hour'], how='left')
synthetic_manpower.drop(columns=['Hour'], inplace=True)
synthetic_manpower.rename(columns={'Operator': 'operator', 'Helper': 'helper', 'Man power': 'manpower'}, inplace=True)
synthetic_manpower

,evt_gmt,Date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,station_id,station,hsk,buf,apt,stp,stpv,operator,helper,manpower
0,2024-06-01 07:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
1,2024-06-01 08:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
2,2024-06-01 09:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1220,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
3,2024-06-01 10:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
4,2024-06-01 11:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
5,2024-06-01 13:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1219,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
6,2024-06-01 14:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1220,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
7,2024-06-01 15:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1219,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
8,2024-06-01 16:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1218,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28
9,2024-06-01 17:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,EQF1219,Station 2,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28


In [102]:
# Add missing column and values if not present in target
if "iron man" not in synthetic_manpower.columns:
    synthetic_manpower["ironman"] = 0
if "qc" not in synthetic_manpower.columns:
    synthetic_manpower["qc"] = 0

In [103]:
synthetic_manpower.rename(columns={'Date': 'factory_date'}, inplace=True)

In [104]:
synthetic_manpower

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,operator,helper,manpower,ironman,qc
0,2024-06-01 07:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
1,2024-06-01 08:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
2,2024-06-01 09:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
3,2024-06-01 10:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
4,2024-06-01 11:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
5,2024-06-01 13:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
6,2024-06-01 14:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
7,2024-06-01 15:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
8,2024-06-01 16:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
9,2024-06-01 17:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0


In [105]:
# edit bctx and values as needed
editable_columns = ['shift_id', 'shift', 'tenant_id', 'tenant',
       'cluster_id', 'cluster', 'factory_id', 'factory', 'building_id',
       'building', 'department_id', 'department', 'floor_id', 'floor',
       'zone_id', 'zone', 'section_id', 'section', 'module_id', 'module',
       ]

edit_df_bctx_org(synthetic_manpower, editable_columns)

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,operator,helper,manpower,ironman,qc
0,2024-06-01 07:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
1,2024-06-01 08:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
2,2024-06-01 09:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
3,2024-06-01 10:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
4,2024-06-01 11:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
5,2024-06-01 13:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
6,2024-06-01 14:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
7,2024-06-01 15:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
8,2024-06-01 16:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0
9,2024-06-01 17:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,23,5,28,0,0


In [106]:
manpower

,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,manpower,operator,helper,ironman,qc
0,2024-06-01 02:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,25,23,1,1,0


In [107]:
# Order the columns
synthetic_manpower = synthetic_manpower.reindex(columns=manpower.columns)
synthetic_manpower


,evt_gmt,factory_date,shift_id,shift,tenant_id,tenant,cluster_id,cluster,factory_id,factory,...,hsk,buf,apt,stp,stpv,manpower,operator,helper,ironman,qc
0,2024-06-01 07:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
1,2024-06-01 08:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
2,2024-06-01 09:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
3,2024-06-01 10:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
4,2024-06-01 11:30:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
5,2024-06-01 13:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
6,2024-06-01 14:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
7,2024-06-01 15:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
8,2024-06-01 16:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0
9,2024-06-01 17:00:00,2024-06-01,1,Morning,1,mas,1,kreeda,ABC001,ABC Factory,...,primary,out,endline_qc,mas-kreeda-intimo-building1-sewing-floor1-zone...,1.0.0,28,23,5,0,0


In [108]:
synthetic_manpower.to_csv('synthetic_manpower_regulated.csv', index=False)